In [ ]:
import csv
import os

import matplotlib.pyplot as plt
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from PIL import Image
from mediapipe.framework.formats.landmark_pb2 import NormalizedLandmarkList

In [ ]:
def get_facemesh_coords(
    landmark_list: NormalizedLandmarkList, img: np.ndarray, normalize: bool = True
) -> np.ndarray:
    """Extract FaceMesh landmark coordinates into 468x3 NumPy array."""
    h, w = img.shape[:2]  # grab width and height from image
    xyz = [(lm.x, lm.y, lm.z) for lm in landmark_list.landmark]
    if normalize:
        return np.array(xyz).astype(float)
    return np.multiply(xyz, [w, h, w]).astype(int)

In [ ]:
data_dir = Path("../data/photographed_data")
visualize_dir = Path("../data/mediapipe_visualize")
meshrender_dir = Path("../data/mediapipe_render")
if not visualize_dir.exists(): visualize_dir.mkdir()
if not meshrender_dir.exists(): meshrender_dir.mkdir()

is_save_image = False
is_render_mesh = False
data_table = pd.read_csv("../data/photographed_data/processed_data_table.csv")

left_iris_index = {474,475, 476, 477}
right_iris_index = {469, 470, 471, 472}
left_eye_index = {362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 }
right_eye_index = {33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 }
facemesh_index = {_ for _ in range(478)}-left_iris_index-right_iris_index-left_eye_index-right_eye_index

iris_idx = sorted(list(left_iris_index | right_iris_index))
eye_idx = sorted(list(left_eye_index | right_eye_index))
face_idx = sorted(list(facemesh_index))

iris_columns = [f"iris{e}_{c}" for e in iris_idx for c in ["x", "y", "z"]]
eye_columns = [f"eye{e}_{c}" for e in eye_idx for c in ["x", "y", "z"]]
face_columns = [f"face{e}_{c}" for e in face_idx for c in ["x", "y", "z"]]

print(len(iris_columns) + len(eye_columns) + len(face_columns))

In [ ]:

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh
with mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
    ) as face_mesh:
    face_mesh_table = np.zeros(shape=(len(data_table), 478*3))
    relative_img_path_list = []
    for i, relative_img_path in enumerate(tqdm(data_table["PATH"].values)):
        mesh_point_cnt = 0
        relative_img_path_list.append(relative_img_path)
        # image = np.array(Image.open("../datasets/photographed_data/C/WIN_20221122_16_15_25_Pro_00000060.jpg"))
        img_path = data_dir.joinpath(relative_img_path)
        image = np.array(Image.open(img_path))
        render = np.zeros(image.shape, dtype=np.uint8)
        results = face_mesh.process(image)
        if results.multi_face_landmarks is not None:
            face_landmarks = results.multi_face_landmarks[0]
            coords = get_facemesh_coords(face_landmarks, image, normalize=False)
            for e in sorted(list(left_iris_index | right_iris_index)):
                landmark = face_landmarks.landmark[e]
                # print(mesh_point_cnt*3 + 0)
                face_mesh_table[i, mesh_point_cnt*3 + 0] = landmark.x
                face_mesh_table[i, mesh_point_cnt*3 + 1] = landmark.y
                face_mesh_table[i, mesh_point_cnt*3 + 2] = landmark.z

                if is_save_image:
                    image = cv2.circle(
                                image, (coords[e, 0], coords[e, 1]), 3, (255, 0, 0), thickness=-1
                            )
                if is_render_mesh:
                    render = cv2.circle(
                                render, (coords[e, 0], coords[e, 1]), 3, (255, 0, 255), thickness=-1
                            )
                mesh_point_cnt += 1
            for e in sorted(list(left_eye_index | right_eye_index)):
                landmark = face_landmarks.landmark[e]
                face_mesh_table[i, mesh_point_cnt*3 + 0] = landmark.x
                face_mesh_table[i, mesh_point_cnt*3 + 1] = landmark.y
                face_mesh_table[i, mesh_point_cnt*3 + 2] = landmark.z

                if is_save_image:
                    image = cv2.circle(
                        image, (coords[e, 0], coords[e, 1]), 2, (0, 255, 0)
                    )
                if is_render_mesh:
                    render = cv2.circle(
                        render, (coords[e, 0], coords[e, 1]), 2, (0, 255, 0)
                    )
                mesh_point_cnt += 1
            for e in sorted(list(facemesh_index)):
                landmark = face_landmarks.landmark[e]
                face_mesh_table[i, mesh_point_cnt*3 + 0] = landmark.x
                face_mesh_table[i, mesh_point_cnt*3 + 1] = landmark.y
                face_mesh_table[i, mesh_point_cnt*3 + 2] = landmark.z
                
                if is_save_image:
                    image = cv2.circle(
                        image, (coords[e, 0], coords[e, 1]), 1, (0, 0, 100)
                    )
                if is_render_mesh:
                    render = cv2.circle(
                        render, (coords[e, 0], coords[e, 1]), 1, (0, 179, 239)
                    )
                mesh_point_cnt += 1
        else:
            # print("Detection Failure")
            pass
        if not visualize_dir.joinpath(relative_img_path).parent.exists():
            visualize_dir.joinpath(relative_img_path).parent.mkdir()
        if not meshrender_dir.joinpath(relative_img_path).parent.exists():
            meshrender_dir.joinpath(relative_img_path).parent.mkdir()
        if is_save_image: Image.fromarray(image).save(visualize_dir.joinpath(relative_img_path))
        if is_render_mesh: Image.fromarray(render).save(meshrender_dir.joinpath(relative_img_path))
        # break
    

    face_mesh_df = pd.DataFrame(face_mesh_table, index=relative_img_path_list, columns=iris_columns+eye_columns+face_columns)
    face_mesh_df.to_csv(visualize_dir.joinpath("facemesh.csv"))

# plt.imshow(image)
# plt.show()
# image.shape

In [ ]:
face_mesh_df.reset_index().rename(columns={"index": "PATH"})

In [ ]:
cat_df = pd.merge(data_table,face_mesh_df.reset_index().rename(columns={"index": "PATH"}))
cat_df.to_csv(data_dir.joinpath("facemesh.csv"), index=None)

In [ ]:
import pandas as pd
pd.read_csv("../data/photographed_data/facemesh.csv")